In [2]:
import pandas as pd
import numpy as np
import ast
import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2

In [3]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

### Reading_Data

In [22]:
review_data = pd.read_csv("../data/yelp_academic_dataset_review.csv")

In [23]:
review_data.head(n=5)

,funny,user_id,review_id,text,business_id,stars,date,useful,type,cool
0,0,KpkOkG6RIf4Ra25Lhhxf1A,NxL8SIC5yqOdnlXCg18IBg,If you enjoy service by someone who is as comp...,2aFiy99vNLklCx3T_tGS9A,5,2011-10-10,0,review,0
1,0,bQ7fQq1otn9hKX-gXRsrgA,pXbbIgOXvLuTi_SPs1hQEQ,After being on the phone with Verizon Wireless...,2aFiy99vNLklCx3T_tGS9A,5,2010-12-29,1,review,0
2,0,r1NUhdNmL6yU9Bn-Yx6FTw,wslW2Lu4NYylb1jEapAGsw,Great service! Corey is very service oriented....,2aFiy99vNLklCx3T_tGS9A,5,2011-04-29,0,review,0
3,0,aW3ix1KNZAvoM8q-WghA3Q,GP6YEearUWrzPtQYSF1vVg,Highly recommended. Went in yesterday looking ...,2LfIuF3_sX6uwe-IR-P0jQ,5,2014-07-14,0,review,1
4,0,YOo-Cip8HqvKp_p9nEGphw,25RlYGq2s5qShi-pn3ufVA,I walked in here looking for a specific piece ...,2LfIuF3_sX6uwe-IR-P0jQ,4,2014-01-15,0,review,0


In [24]:
business_data = pd.read_csv("../data/yelp_academic_dataset_business.csv") 

In [25]:
business_data.shape

(144072, 16)

In [26]:
business_data.head(n=5)

,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,name,longitude,state,stars,latitude,postal_code,type
0,NaN,0DI8Dt2PJp07XkVvIElIcQ,"['Monday 11:0-21:0', 'Tuesday 11:0-21:0', 'Wed...",0,"227 E Baseline Rd, Ste J2","['BikeParking: True', 'BusinessAcceptsBitcoin:...","['Tobacco Shops', 'Nightlife', 'Vape Shops', '...",Tempe,17,Innovative Vapors,-111.936102,AZ,4.5,33.378214,85283,business
1,NaN,LTlCaCGZE14GuaUXUGbamg,"['Monday 0:0-0:0', 'Tuesday 0:0-0:0', 'Wednesd...",1,495 S Grand Central Pkwy,"['BusinessAcceptsBitcoin: False', 'BusinessAcc...","['Caterers', 'Grocery', 'Food', 'Event Plannin...",Las Vegas,9,Cut and Taste,-115.159272,NV,5.0,36.192284,89106,business
2,Dufferin Grove,EDqCEAGXVGCH4FJXgqtjqg,"['Monday 11:0-2:0', 'Tuesday 11:0-2:0', 'Wedne...",1,979 Bloor Street W,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Pizza', 'Chicken Wings', 'Ita...",Toronto,7,Pizza Pizza,-79.429089,ON,2.5,43.661054,M6H 1L5,business
3,NaN,cnGIivYRLxpF7tBVR_JwWA,"['Tuesday 10:0-21:0', 'Wednesday 10:0-21:0', '...",1,7014 Steubenville Pike,"['AcceptsInsurance: False', 'BusinessAcceptsCr...","['Hair Removal', 'Beauty & Spas', 'Blow Dry/Ou...",Oakdale,4,Plush Salon and Spa,-80.174540,PA,4.0,40.444544,15071,business
4,Downtown Core,cdk-qqJ71q6P7TJTww_DSA,NaN,1,321 Jarvis Street,"['BusinessAcceptsCreditCards: True', 'Restaura...","['Hotels & Travel', 'Event Planning & Services...",Toronto,8,Comfort Inn,-79.375401,ON,3.0,43.659829,M5B 2C2,business


### Getting Review Data for only Restaurants (Pittsburgh)

In [27]:
business_data = business_data.loc[business_data["categories"].notnull(),:]

In [28]:
def convert_list(x):
    return ast.literal_eval(x)
business_data["categories"] = map(convert_list,business_data["categories"])

In [19]:
category_labels = []
for categories in business_data["categories"]:
    category_labels += categories


In [30]:
category_dict = Counter(category_labels)
#categorstaurants

In [31]:
Restaurant = [1 if "Restaurants" in categories else 0 for categories in business_data["categories"]]
business_data["Restaurant"] = Restaurant
np.sum(Restaurant)
float(np.sum(Restaurant))/business_data.shape[0]

0.33729399570077984

In [32]:
restaurant_data = business_data.loc[business_data["Restaurant"]==1,:]
restaurant_review = pd.merge(review_data,restaurant_data,on="business_id")

In [34]:
dump(restaurant_review,"../Pickled_files/restaurant_review.pkl")

### Load Data Start from here

In [4]:
restaurant_review = load("../Pickled_files/restaurant_review.pkl")


In [5]:
restaurant_review.head(n=5)

,funny,user_id,review_id,text,business_id,stars_x,date,useful,type_x,cool,...,city,review_count,name,longitude,state,stars_y,latitude,postal_code,type_y,Restaurant
0,0,PmgqNO0-5Y3e3UoR61TD7w,6-hKBi-6RC3g7Mft0c-6qw,This place is a area staple! Been around for y...,4P-vTvE6cncJyUyLh73pxw,4,2014-03-29,0,review,0,...,Brecksville,23,London Pickle Works,-81.627,OH,3.5,41.340319,44141,business,1
1,0,kXUySHSlRgVrcR4Aa0HtGQ,tRd0-mPa9O1TMJp_dw5khQ,Got my mojo back after having a few of their a...,4P-vTvE6cncJyUyLh73pxw,4,2014-08-14,0,review,0,...,Brecksville,23,London Pickle Works,-81.627,OH,3.5,41.340319,44141,business,1
2,0,SYKPwRhnlKrW6yTvm7oPWg,8Mu56iQ-MYEyivqUVss0XA,"Don't go here for the decor, but the staff is ...",4P-vTvE6cncJyUyLh73pxw,4,2012-10-20,1,review,0,...,Brecksville,23,London Pickle Works,-81.627,OH,3.5,41.340319,44141,business,1
3,0,7LCG3o2KW2jgKgbKN0DQOg,ToC77cIEiMas9CPU7dt_fA,I believe in awarding stars bearing in mind th...,4P-vTvE6cncJyUyLh73pxw,5,2012-08-25,2,review,1,...,Brecksville,23,London Pickle Works,-81.627,OH,3.5,41.340319,44141,business,1
4,0,iSdSNh1hjdE33LOwrFnFrg,6YC4o9yLc25DK8c6soOlaw,"If you like fried food and laid back, then thi...",4P-vTvE6cncJyUyLh73pxw,4,2015-07-27,0,review,0,...,Brecksville,23,London Pickle Works,-81.627,OH,3.5,41.340319,44141,business,1


### Add your city and get the data

In [7]:

restaurant_review["city"].values

array(['Brecksville', 'Brecksville', 'Brecksville', ..., 'Las Vegas',
       'Las Vegas', 'Las Vegas'], dtype=object)

In [9]:
cities = set(restaurant_review["city"])
cities

{'AGINCOURT',
 'Ahwatukee',
 'Aichwald',
 'Ajax',
 'Allison Park',
 'Ambridge',
 'Amherst',
 'Anjou',
 'Anthem',
 'Arnold',
 'Aspinwall',
 'Auburn Township',
 'Aurora',
 'Avalon',
 'Avon',
 'Avon Lake',
 'Avondale',
 "Baie-D'urfe",
 'Bainbridge',
 'Bainbridge Township',
 'Baldwin',
 'Balerno',
 'Bath',
 'Bathurst Quay',
 'Bay Village',
 'Beachwood',
 'Beaconsfield',
 'Bedford',
 'Bedford Heights',
 'Bedford Hts.',
 'Beeton',
 'Belleville',
 'Bellevue',
 'Bellvue',
 'Belmont',
 'Beloeil',
 'Ben Avon',
 'Berea',
 'Berg',
 'Bernhausen',
 'Bethel Park',
 'Black Earth',
 'Blainville',
 'Blawnox',
 'Bloomfield',
 'Blue Diamond',
 'Bois-Des-Filion',
 'Boisbriand',
 'Bolton',
 'Bonnyrigg and Lasswade',
 'Boucherville',
 'Boulder City',
 'Braddock',
 'Bradford',
 'Bradford West Gwillimbury',
 'Brampton',
 'Bratenahl',
 'Brecksville',
 'Brentwood',
 'Bridgeville',
 'Broadview Heights',
 'Brook Park',
 'Brooklin',
 'Brooklyn',
 'Brookpark',
 'Brossard',
 'Brunswick',
 'Brunswick Hills',
 'Bruntsf

In [ ]:
#pitt_review = restaurant_review[restaurant_review["city"]=="Pittsburgh"]


In [54]:
montreal_review_1 = restaurant_review[restaurant_review["city"]=="Montreal"]
montreal_review_2 = restaurant_review[restaurant_review["city"]=="Montreal-Nord"]
montreal_review_3 = restaurant_review[restaurant_review["city"]=="Montreal-Ouest"]
montreal_review_4 = restaurant_review[restaurant_review["city"]=="Montreal-West"]
montreal_review_5 = restaurant_review[restaurant_review["city"]=="Montr\xc3\xa9al"]
montreal_review_6 = restaurant_review[restaurant_review["city"]=="Montr\xc3\xa9al-Nord"]
montreal_review_7 = restaurant_review[restaurant_review["city"]=="Montr\xc3\xa9al-Ouest"]
montreal_review_8 = restaurant_review[restaurant_review["city"]=="Mont\xc3\xa9al"]


In [55]:
montreal_review = pd.concat([montreal_review_1, montreal_review_2, montreal_review_3, montreal_review_4, montreal_review_5, montreal_review_6, montreal_review_7, montreal_review_8])

In [56]:
edinburgh_review_1 = restaurant_review[restaurant_review["city"]=="Edinburgh"]
edinburgh_review_2 = restaurant_review[restaurant_review["city"]=="City of Edinburgh"]

In [57]:
type(edinburgh_review_1)
edinburgh_review = pd.concat([edinburgh_review_1, edinburgh_review_2])

In [58]:
brooklyn_review = restaurant_review[restaurant_review["city"]=="Brooklyn"]

In [59]:
toronto_review_1 = restaurant_review[restaurant_review["city"]=="TORONTO"]
toronto_review_2 = restaurant_review[restaurant_review["city"]=="Toronto"]
toronto_review = pd.concat([toronto_review_1, toronto_review_2])

### Clean Reviews

In [36]:
import nltk
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
def preprocess_text(text):
    parse_text = BeautifulSoup(text).get_text()
    tokens = CountVectorizer(stop_words='english').build_tokenizer()(parse_text.lower())
    result = []
    
    
    for w in tokens:
        if w in stops:
            continue
        lem_word = wordnet_lemmatizer.lemmatize(w)
        stem_word = snowball_stemmer.stem(lem_word)
        result.append(stem_word)
    
    
    
    return result

In [37]:
#restaurant_review["clean_text"] = map(preprocess_text,restaurant_review["text"])

### Map the function to the data of the city

In [73]:
montreal_review["clean_text"] = map(preprocess_text,montreal_review["text"])

In [70]:
edinburgh_review["clean_text"] = map(preprocess_text,edinburgh_review["text"])

In [64]:
brooklyn_review["clean_text"] = map(preprocess_text,brooklyn_review["text"])

/Users/YJChang/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
toronto_review["clean_text"] = map(preprocess_text,toronto_review["text"])

In [74]:
montreal_review.shape

(75664, 27)

In [71]:
edinburgh_review.shape

(23794, 27)

In [65]:
brooklyn_review.shape

(625, 27)

In [68]:
toronto_review.shape

(245131, 27)

In [75]:
dump(montreal_review,"../Pickled_files/montreal_review.pkl")

In [72]:
dump(edinburgh_review,"../Pickled_files/edinburgh_review.pkl")

In [66]:
dump(brooklyn_review,"../Pickled_files/brooklyn_review.pkl")

In [69]:
dump(toronto_review,"../Pickled_files/toronto_review.pkl")

In [ ]:
pitt_review = load("../Pickled_files/pitt_review.pkl")

In [ ]:
edinburgh_review = load("../Pickled_files/edinburgh_review.pkl")

In [ ]:
brooklyn_review = load("../Pickled_files/brooklyn_review.pkl")

In [ ]:
toronto_review = load("../Pickled_files/toronto_review.pkl")

In [ ]:
montreal_review = load("../Pickled_files/montreal_review.pkl")

### Preparing data for gensim for LDA

In [ ]:
data = pitt_review["clean_text"]
dictionary = gensim.corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]

In [77]:
data_montreal = montreal_review["clean_text"]
dictionary_montreal = gensim.corpora.Dictionary(data_montreal)
corpus_montreal = [dictionary_montreal.doc2bow(text) for text in data_montreal]

In [81]:
data_edinburgh = edinburgh_review["clean_text"]
dictionary_edinburgh = gensim.corpora.Dictionary(data_edinburgh)
corpus_edinburgh = [dictionary_edinburgh.doc2bow(text) for text in data_edinburgh]

In [83]:
data_brooklyn = brooklyn_review["clean_text"]
dictionary_brooklyn = gensim.corpora.Dictionary(data_brooklyn)
corpus_brooklyn = [dictionary_brooklyn.doc2bow(text) for text in data_brooklyn]

In [84]:
data_toronto = toronto_review["clean_text"]
dictionary_toronto = gensim.corpora.Dictionary(data_toronto)
corpus_toronto = [dictionary_toronto.doc2bow(text) for text in data_toronto]

In [80]:
dump(corpus_montreal,"../Pickled_files/corpus_montreal.pkl")
dump(dictionary_montreal,"../Pickled_files/dictionary_montreal.pkl")

In [82]:
dump(corpus_edinburgh,"../Pickled_files/corpus_edinburgh.pkl")
dump(dictionary_edinburgh,"../Pickled_files/dictionary_edinburgh.pkl")

In [85]:
dump(corpus_toronto,"../Pickled_files/corpus_toronto.pkl")
dump(dictionary_toronto,"../Pickled_files/dictionary_toronto.pkl")

In [86]:
dump(corpus_brooklyn,"../Pickled_files/corpus_brooklyn.pkl")
dump(dictionary_brooklyn,"../Pickled_files/dictionary_brooklyn.pkl")

In [ ]:
dump(corpus,"../Pickled_files/corpus.pkl")
dump(dictionary,"../Pickled_files/dictionary.pkl")

### Checkin Data 

In [ ]:
checkin_data = pd.read_csv("../data/yelp_academic_dataset_checkin.csv")

In [ ]:
len(set(checkin_data["business_id"]))

In [ ]:
pitt_review = load("../Pickled_files/pitt_review.pkl")

In [ ]:
p